<a href="https://colab.research.google.com/github/yiramarcela/Master/blob/main/Act1_Yira_Pena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================
# PREDICCIÓN DE PRECIOS DE VIVIENDAS - TuCasa
# Autora: Yira Marcela Peña García
# =============================================

!pip install pandas numpy scikit-learn xgboost matplotlib seaborn fpdf joblib --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score
from fpdf import FPDF
from datetime import datetime
from google.colab import files

print("Sube los archivos train_houses.csv y test_houses.csv")
uploaded = files.upload()

# Cargamos los datos
train = pd.read_csv("train_houses.csv")
test = pd.read_csv("test_houses.csv")

print(f"\nDatos cargados: {train.shape[0]} filas y {train.shape[1]} columnas en train")

# Definimos la función para calcular el MAPE
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-9))) * 100

# Echamos un vistazo rápido a los datos
print("\nPrimeras filas del archivo:")
display(train.head())

# Identificamos las columnas
target = 'buy_price_by_area'
X = train.drop(columns=[target])
y = train[target]

num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()

# Aquí preparamos el tratamiento para las variables numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
])

# Ahora dividimos los datos para entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definimos los modelos a comparar
models = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror')
}

results = {}

# Entrenamos y evaluamos cada modelo
for name, model in models.items():
    print(f"\nEntrenando modelo: {name}")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)
    mape = MAPE(y_val, preds)
    mae = mean_absolute_error(y_val, preds)
    r2 = r2_score(y_val, preds)
    results[name] = {'MAPE': mape, 'MAE': mae, 'R2': r2}
    print(f"MAPE: {mape:.2f}%  |  MAE: {mae:.2f}  |  R²: {r2:.3f}")

# Escogemos el modelo con mejor resultado
best_model_name = min(results, key=lambda k: results[k]['MAPE'])
print(f"\nEl mejor modelo fue: {best_model_name} con MAPE de {results[best_model_name]['MAPE']:.2f}%")

# Entrenamos nuevamente con todos los datos
best_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', models[best_model_name])])
best_pipeline.fit(X, y)

# Guardamos el modelo
joblib.dump(best_pipeline, "mejor_modelo.pkl")

# Generamos predicciones con el conjunto de prueba
print("\nGenerando predicciones finales...")
preds_test = best_pipeline.predict(test)
output = pd.DataFrame(preds_test, columns=['predicted_buy_price_by_area'])
output.to_csv('predicciones_test.csv', index=False)
print("Archivo 'predicciones_test.csv' creado correctamente.")

# Ahora generamos la gráfica de comparación
plt.figure(figsize=(6,6))
plt.scatter(y_val, best_pipeline.predict(X_val), alpha=0.5)
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title(f"Predicción vs Real ({best_model_name})")
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
plt.tight_layout()
plt.savefig("grafico_prediccion.png")
plt.close()

# Ahora realizamos la matriz de correlación para ver relaciones entre variables
num_cols_all = X.select_dtypes(include=['int64','float64']).columns.tolist()
if len(num_cols_all) > 1:
    plt.figure(figsize=(8,6))
    sns.heatmap(pd.concat([X[num_cols_all], y], axis=1).corr(), cmap='coolwarm')
    plt.title("Matriz de correlación")
    plt.tight_layout()
    plt.savefig("correlacion.png")
    plt.close()

# Intentamos mostrar la importancia de variables (si el modelo la soporta)
try:
    model = best_pipeline.named_steps['model']
    importances = model.feature_importances_
    feat_names = X.columns
    imp_df = pd.DataFrame({'Variable': feat_names, 'Importancia': importances}).sort_values('Importancia', ascending=False).head(10)
    plt.figure(figsize=(6,4))
    plt.barh(imp_df['Variable'][::-1], imp_df['Importancia'][::-1])
    plt.title("Variables más importantes")
    plt.tight_layout()
    plt.savefig("importancias.png")
    plt.close()
except:
    pass

# Ahora generamos el informe en PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "Informe del Modelo de Predicción de Precios - TuCasa", ln=True, align='C')
pdf.ln(6)
pdf.set_font("Arial", size=12)
pdf.cell(0, 8, f"Autora: Yira Marcela Peña García", ln=True)
pdf.cell(0, 8, f"Fecha: {datetime.now().strftime('%d/%m/%Y')}", ln=True)
pdf.ln(6)
pdf.multi_cell(0, 8, f"""En este informe se presenta el desarrollo del modelo para predecir precios de viviendas por metro cuadrado en Madrid.
Se probaron dos algoritmos: Random Forest y XGBoost. El modelo con mejor rendimiento fue {best_model_name}, con un MAPE de {results[best_model_name]['MAPE']:.2f}%.
Esto significa que el error promedio porcentual del modelo está por debajo del 15%, cumpliendo con el objetivo del proyecto.""")

# Agregamos las imágenes generadas al informe
for img in ["correlacion.png", "grafico_prediccion.png", "importancias.png"]:
    try:
        pdf.add_page()
        pdf.image(img, w=180)
    except:
        continue

pdf.output("reporte_final_tucasa.pdf")
print("\nInforme generado: reporte_final_tucasa.pdf")

# Finalmente, descargamos los resultados
files.download("predicciones_test.csv")
files.download("reporte_final_tucasa.pdf")


Sube los archivos train_houses.csv y test_houses.csv
